In [15]:
from __future__ import print_function
import time
import os

dataDir = '/Users/manojthalari/Stevens/Data/music/'
TRACK_DATA_FILE = os.path.join(dataDir, 'trackData2.txt')
TRACK_HIERARCHY_FILE = os.path.join(dataDir, 'testTrack_hierarchy.txt')
TEST_DATA_FILE = os.path.join(dataDir, 'testItem2.txt')

lib_trackData = {}
start_time = time.time()

# Read track data into a dictionary
with open(TRACK_DATA_FILE) as trackData:
    for line in trackData:
        track_Id, track_detail = line.strip().split("|", maxsplit=1)
        lib_trackData[track_Id] = track_detail

# Create the hierarchy file
with open(TRACK_HIERARCHY_FILE, "w") as testHierarchy, open(TEST_DATA_FILE) as testData:
    cur_user = None  # Initialize cur_user
    for line in testData:
        if "|" in line:
            cur_user = line.strip().split("|")[0]
            continue
        cur_track = line.strip()
        # Use get method to handle the case where cur_track is not in lib_trackData
        track_info = lib_trackData.get(cur_track, 'Unknown Track')
        testHierarchy.write(f"{cur_user}|{cur_track}|{track_info}\n")

print(f"Finished, Spend {time.time() - start_time:.2f} seconds.")


Finished, Spend 0.16 seconds.


In [16]:
from __future__ import print_function
import time
import os

dataDir = '/Users/manojthalari/Stevens/Data/music/'
OUTPUT_FILE = os.path.join(dataDir, 'Ranking2.txt')
H_FILE = os.path.join(dataDir, 'testTrack_hierarchy.txt')
TRAIN_FILE = os.path.join(dataDir, 'trainItem2.txt')

user_rate = {}
start_time = time.time()

# Read user ratings from train file into a dictionary
with open(TRAIN_FILE) as train:
    for line in train:
        if "|" in line:
            cur_user = line.strip("\n").split("|")[0]
            user_rate[cur_user] = {}
            continue
        item_id, item_score = line.strip("\n").split()
        user_rate[cur_user][item_id] = item_score

# Write output to the Ranking2.txt file
with open(OUTPUT_FILE, "w") as output, open(H_FILE) as record:
    for line in record:
        gen_out = ""
        user, track, *items = line.strip("\n").split("|")
        if not items:
            album_score, artist_score = '0', '0'
        elif len(items) == 1:
            album = items[0]
            album_score = user_rate[user].get(album, '0')
            artist_score = '0'
        elif len(items) == 2:
            album, artist = items
            album_score = user_rate[user].get(album, '0')
            artist_score = user_rate[user].get(artist, '0')
        else:
            album_score = user_rate[user].get(items[0], '0')
            artist_score = user_rate[user].get(items[1], '0')
            genr = items[2:]
            for g in genr:
                gen_out += "|" + user_rate[user].get(g, '0')

        output.write(f"{user}|{track}|{album_score}|{artist_score}{gen_out}\n")

print(f"Finished, Spend {time.time() - start_time:.2f} seconds.")


Finished, Spend 5.10 seconds.


In [17]:
import time
import os

dataDir = '/Users/manojthalari/Stevens/Data/music/'
SCORE_FILE = os.path.join(dataDir, 'Ranking2.txt')
OUT_FILE = os.path.join(dataDir, 'Ranking3.txt')

user_rate = {}
start_time = time.time()

def calculate_weighted_sum(scores):
    sum_score = (1 * int(scores[0]) + 0.01 * int(scores[1]) + 0.00007 * my_sum(scores[2:])) / 3
    return sum_score

def my_sum(lst):
    non_zero_values = [int(i) for i in lst if int(i) != 0]
    count = len(non_zero_values)
    if count != 0:
        return sum(non_zero_values) / count
    else:
        return sum(non_zero_values)

with open(OUT_FILE, 'w') as output:
    with open(SCORE_FILE) as score:
        for line in score:
            try:
                user_id, track_id, sum_score = line.strip("\n").split("|")[:3]
                extra_scores = line.strip("\n").split("|")[3:]
                sum_score = calculate_weighted_sum([int(sum_score)] + [int(score) for score in extra_scores])
            except IndexError:
                user_id, track_id, sum_score = line.strip("\n").split("|")[:3]
            output.write(f"{user_id}|{track_id}|{sum_score}\n")

print(f"Finished, Spend {time.time() - start_time:.2f} seconds.")


Finished, Spend 0.28 seconds.


In [18]:
SUM_FILE = os.path.join(dataDir, "Ranking3.txt")
OUT_FILE = os.path.join(dataDir, "Compare7.txt")

with open(OUT_FILE, "w") as output:
    with open(SUM_FILE) as score:
        cur = 0
        vec = {}
        for line in score:
            user_track = line.strip('\n').split('|')[0] + "_" + line.strip('\n').split('|')[1]
            vec[user_track] = line.strip("\n").split("|")[2]
            cur += 1

            if cur == 6:
                sorted_items = sorted(vec.items(), key=lambda item: item[1])

                for i in range(3):
                    if i < len(sorted_items):
                        output.write(sorted_items[i][0] + "|0\n")
                    else:
                        output.write("")

                for j in range(3, 6):
                    if j < len(sorted_items):
                        output.write(sorted_items[j][0] + "|1\n")
                    else:
                        output.write("")

                vec = {}
                cur = 0

        sorted_items = sorted(vec.items(), key=lambda item: item[1])

        for i in range(3):
            if i < len(sorted_items):
                output.write(sorted_items[i][0] + "|0\n")
            else:
                output.write("")

        for j in range(3, 6):
            if j < len(sorted_items):
                output.write(sorted_items[j][0] + "|1\n")
            else:
                output.write("")

print(f"Finished, Spend {time.time() - start_time:.2f} seconds.")


Finished, Spend 0.42 seconds.


In [19]:
import pandas as pd

OUT_FILE = os.path.join(dataDir, "Compare7.txt")
result_list = []

with open(OUT_FILE, "r") as output:
    for line in output:
        line = line.strip().split("|")
        ID = line[0]
        score = line[1]
        result = [ID, score]
        result_list.append(result)

column_names = ['TrackID', 'Predictor']
result_df = pd.DataFrame(result_list, columns=column_names)
result_df.to_csv(os.path.join(dataDir, 'result3.csv'), index=False)
